<br>

# Introdução

In [ ]:
import os
import pprint
import pkg_resources
import pandas as pd

In [ ]:
def print_michel():
    print(__name__)
    return print('Caralha de pasta data')

print_michel()

In [ ]:
def get_8468_parameters():
    # Read Data
    df_8468 = pd.read_excel(
        #io='https://raw.githubusercontent.com/gaemapiracicaba/norma_dec_8468-76/main/data/tab_dec_8468.xlsx',
        #io=os.path.join(os.path.dirname(__file__), 'data', 'tab_dec_8468.xlsx'),
        io=pkg_resources.resource_filename(__name__, 'data/tab_dec_8468.xlsx'),
        sheet_name='dec_8468',
        index_col=0
    )

    # Filter only quality
    df_8468 = df_8468.loc[(df_8468['tipo_padrao'] == 'qualidade')]
    #print(df_8468.head())

    # Classes
    list_classes = list(set(df_8468['padrao_qualidade']))
    list_classes = [x for x in list_classes if pd.notnull(x)]
    list_classes.sort()
    pprint.pprint(list_classes)

    return df_8468, list_classes

In [ ]:
df_8468, list_classes = get_8468_parameters()

In [ ]:
def filter_by_classe(df_8468, classe):
    # Filter dataframe by Classe
    df_8468 = df_8468.loc[(df_8468['padrao_qualidade'] == classe)]

    # Parâmetros
    list_parametros = list(set(df_8468['parametro_descricao']))
    list_parametros = [x for x in list_parametros if pd.notnull(x)]
    list_parametros.sort()
    pprint.pprint(list_parametros)
    return df_8468, list_parametros

In [ ]:
df_8468, list_parametros = filter_by_classe(df_8468, classe='Classe 2')

In [ ]:
def filter_by_parameters(df_8468, parametro):
    # Filter dataframe by Parametro
    df_8468 = df_8468.loc[(df_8468['parametro_descricao'] == parametro)]

    # Check and Get Results
    if len(df_8468) == 1:
        dict_8468 = df_8468.to_dict(orient='records')[0]
        return dict_8468
    else:
        print('Erro')

In [ ]:
# Filter Data by Parâmetros
dict_8468 = filter_by_parameters(df_8468, parametro='Oxigênio Dissolvido')
dict_8468

In [ ]:
def set_type_desconformidade(dict_8468):
    if pd.isnull(dict_8468['valor_minimo_permitido']) & pd.isnull(dict_8468['valor_maximo_permitido']):
        print('Erro!')
        tipo_8486 = 'erro'

    elif pd.isnull(dict_8468['valor_minimo_permitido']) & pd.notnull(dict_8468['valor_maximo_permitido']):
        print('Parâmetro só tem "valor máximo". Caso o valor medido esteja acima, é amostra desconforme!')
        tipo_8486 = 'acima>desconforme'

    elif pd.notnull(dict_8468['valor_minimo_permitido']) & pd.isnull(dict_8468['valor_maximo_permitido']):
        print('Parâmetro só tem "valor mínimo". Caso o valor medido esteja abaixo, é amostra desconforme!')
        tipo_8486 = 'abaixo>desconforme'

    elif pd.notnull(dict_8468['valor_minimo_permitido']) & pd.notnull(dict_8468['valor_maximo_permitido']):
        print('Parâmetro tem "valor mínimo" e "valor máximo". Caso o valor medido acima ou abaixo, é amostra desconforme!')
        tipo_8486 = 'abaixo_acima>desconforme'

    return tipo_8486

In [ ]:
set_type_desconformidade(dict_8468)

<br>

# Export

In [ ]:
import os
from traitlets.config import Config
from nbconvert import PythonExporter
from nbconvert.preprocessors import TagRemovePreprocessor

In [ ]:
input_filename = 'decreto_estadual_8468.ipynb'
input_filepath = os.path.join(os.getcwd(), input_filename)

In [ ]:
# Import the exporter
c = Config()
c.TagRemovePreprocessor.enabled=True
c.ClearOutputPreprocessor.enabled=True
c.TemplateExporter.exclude_markdown=True
c.TemplateExporter.exclude_code_cell=False
c.TemplateExporter.exclude_input_prompt=True
c.TemplateExporter.exclude_output=True
c.TemplateExporter.exclude_raw=True
c.TagRemovePreprocessor.remove_cell_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_input_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_all_outputs_tags = ('remove_output',)
c.preprocessors = ['TagRemovePreprocessor']
c.PythonExporter.preprocessors = ['nbconvert.preprocessors.TagRemovePreprocessor']

# Configure and run out exporter
py_exporter = PythonExporter(config=c)
py_exporter.register_preprocessor(TagRemovePreprocessor(config=c), True)

# Configure and run out exporter - returns a tuple - first element with html, second with notebook metadata
body, metadata = PythonExporter(config=c).from_filename(input_filepath)

# Write to output html file
with open(os.path.join(os.getcwd(), '..', 'src', 'decreto_estadual_8468.py'),  'w', encoding='utf-8') as f:
    f.write(body)